<a href="https://colab.research.google.com/github/anms5519/Handwritten-Digit-Recognizer/blob/main/working3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr opencv-python ipywidgets
import easyocr
import cv2
import numpy as np
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
from base64 import b64decode
bengali_to_english = {
    '০': '0', '১': '1', '২': '2', '৩': '3', '৪': '4',
    '৫': '5', '৬': '6', '৭': '7', '৮': '8', '৯': '9'
}
english_to_bengali = {
    '0': '০', '1': '১', '2': '২', '3': '৩', '4': '৪',
    '5': '৫', '6': '৬', '7': '৭', '8': '৮', '9': '৯'
}
def convert_bengali_to_english(text):
    """Convert Bengali digits to English digits."""
    for bengali, english in bengali_to_english.items():
        text = text.replace(bengali, english)
    return text
def convert_english_to_bengali(text):
    """Convert English digits to Bengali digits."""
    for english, bengali in english_to_bengali.items():
        text = text.replace(english, bengali)
    return text
def process_image_from_array(img_array):
    """
    Process an image from a numpy array:
      - Convert to grayscale
      - Detect text using EasyOCR (supports English and Bengali)
      - Convert digits accordingly and evaluate the equation
    """
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['en', 'bn'])
    result = reader.readtext(gray, detail=0)
    equation = ' '.join(result)
    is_bangla = any(char in bengali_to_english for char in equation)
    equation_converted = convert_bengali_to_english(equation)
    allowed_chars = "0123456789+-*/(). "
    cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
    output_text = (f"Detected Equation: {equation}\n"
                   f"After Conversion: {equation_converted}\n"
                   f"Cleaned Equation: {cleaned_equation}\n")
    try:
        calc_result = eval(cleaned_equation)
        result_str = str(calc_result)
        if is_bangla:
            result_str = convert_english_to_bengali(result_str)
        output_text += f"Result: {result_str}"
    except Exception as e:
        output_text += f"Error evaluating equation: {e}"
    return output_text
def take_photo(filename='photo.jpg', quality=0.8):
    """
    Capture a photo using the webcam via a JavaScript interface.
    The captured image is saved and then read using OpenCV.
    """
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    img = cv2.imread(filename)
    return img
manual_text = widgets.Text(
    value='',
    placeholder='Enter your equation e.g. 2+2 or ২+২',
    description='Equation:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)
manual_output = widgets.Output()
def on_manual_calculate(b):
    with manual_output:
        clear_output()
        equation = manual_text.value
        is_bangla = any(char in bengali_to_english for char in equation)
        equation_converted = convert_bengali_to_english(equation)
        allowed_chars = "0123456789+-*/(). "
        cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
        print(f"Input Equation: {equation}")
        print(f"After Conversion: {equation_converted}")
        print(f"Cleaned Equation: {cleaned_equation}")
        try:
            result = eval(cleaned_equation)
            result_str = str(result)
            if is_bangla:
                result_str = convert_english_to_bengali(result_str)
            print(f"Result: {result_str}")
        except Exception as e:
            print(f"Error evaluating equation: {e}")
manual_button = widgets.Button(description="Calculate", button_style='success')
manual_button.on_click(on_manual_calculate)
manual_box = widgets.VBox([manual_text, manual_button, manual_output])
camera_output = widgets.Output()
def on_camera_capture(b):
    with camera_output:
        clear_output()
        print("Activating camera... Click 'Capture' when ready.")
        img = take_photo()
        if img is not None:
            result_text = process_image_from_array(img)
            print(result_text)
        else:
            print("No image captured.")
camera_button = widgets.Button(description="Capture Equation", button_style='info')
camera_button.on_click(on_camera_capture)
camera_box = widgets.VBox([camera_button, camera_output])
upload_output = widgets.Output()
def on_upload_change(change):
    with upload_output:
        clear_output()
        for filename, file_info in uploader.value.items():
            nparr = np.frombuffer(file_info['content'], np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            result_text = process_image_from_array(img)
            print(f"File: {filename}\n{result_text}\n{'-'*40}")
uploader = widgets.FileUpload(
    accept='image/*',
    multiple=True
)
uploader.observe(on_upload_change, names='value')
upload_box = widgets.VBox([uploader, upload_output])
tab = widgets.Tab(children=[manual_box, camera_box, upload_box])
tab.set_title(0, 'Manual Calculation')
tab.set_title(1, 'Camera Scan')
tab.set_title(2, 'Upload Photo')
display(tab)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
%%writefile legendary_app.py
# pip install kivy easyocr opencv-python numpy
import cv2
import numpy as np
import easyocr
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button
from kivy.uix.textinput import TextInput
from kivy.uix.label import Label
from kivy.uix.popup import Popup
from kivy.uix.filechooser import FileChooserIconView
# Conversion dictionaries for digit transformation
bengali_to_english = {
    '০': '0', '১': '1', '২': '2', '৩': '3', '৪': '4',
    '৫': '5', '৬': '6', '৭': '7', '৮': '8', '৯': '9'
}
english_to_bengali = {
    '0': '০', '1': '১', '2': '২', '3': '৩', '4': '৪',
    '5': '৫', '6': '৬', '7': '৭', '8': '৮', '9': '৯'
}
def convert_bengali_to_english(text):
    for bengali, english in bengali_to_english.items():
        text = text.replace(bengali, english)
    return text
def convert_english_to_bengali(text):
    for english, bengali in english_to_bengali.items():
        text = text.replace(english, bengali)
    return text
def process_image(img_path):
    """Process an image file, detect text, and evaluate the equation."""
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['en', 'bn'])
    result = reader.readtext(gray, detail=0)
    equation = ' '.join(result)
    is_bangla = any(char in bengali_to_english for char in equation)
    equation_converted = convert_bengali_to_english(equation)
    allowed_chars = "0123456789+-*/(). "
    cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
    try:
        calc_result = eval(cleaned_equation)
        result_str = str(calc_result)
        if is_bangla:
            result_str = convert_english_to_bengali(result_str)
        return f"Detected Equation: {equation}\nResult: {result_str}"
    except Exception as e:
        return f"Error evaluating equation: {e}"
class MainWidget(BoxLayout):
    def __init__(self, **kwargs):
        super(MainWidget, self).__init__(orientation='vertical', padding=10, spacing=10, **kwargs)
        self.input_box = TextInput(
            hint_text='Enter equation (e.g. 2+2 or ২+২)',
            size_hint_y=None, height=40
        )
        self.add_widget(self.input_box)
        self.calc_button = Button(
            text='Calculate',
            size_hint_y=None, height=40
        )
        self.calc_button.bind(on_press=self.calculate)
        self.add_widget(self.calc_button)
        self.result_label = Label(
            text='Result will be shown here', halign='center'
        )
        self.add_widget(self.result_label)
        self.upload_button = Button(
            text='Upload Image', size_hint_y=None, height=40
        )
        self.upload_button.bind(on_press=self.open_filechooser)
        self.add_widget(self.upload_button)
    def calculate(self, instance):
        equation = self.input_box.text
        is_bangla = any(char in bengali_to_english for char in equation)
        equation_converted = convert_bengali_to_english(equation)
        allowed_chars = "0123456789+-*/(). "
        cleaned_equation = ''.join([c for c in equation_converted if c in allowed_chars])
        try:
            result = eval(cleaned_equation)
            result_str = str(result)
            if is_bangla:
                result_str = convert_english_to_bengali(result_str)
            self.result_label.text = f"Result: {result_str}"
        except Exception as e:
            self.result_label.text = f"Error: {e}"
    def open_filechooser(self, instance):
        chooser = FileChooserIconView()
        popup = Popup(title="Select an image", content=chooser, size_hint=(0.9, 0.9))
        chooser.bind(on_submit=lambda chooser, selection, touch: self.load_image(selection, popup))
        popup.open()
    def load_image(self, selection, popup):
        if selection:
            result_text = process_image(selection[0])
            self.result_label.text = result_text
        popup.dismiss()
class LegendaryApp(App):
    def build(self):
        return MainWidget()
if __name__ == '__main__':
    LegendaryApp().run()

Writing legendary_app.py


In [ ]:
from google.colab import files
files.download('legendary_app.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>